In [1]:
#import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import tweepy
import re
import csv

# Load data

 The main dataset used in our project is taken from Kaggle: 

In [2]:
#read files 
dcomany = pd.read_csv("data/Company.csv")
dcomany_tweets = pd.read_csv("data/Company_Tweet.csv")
dtweet = pd.read_csv("data/Tweet.csv")


##  Extruct the Number of followers of each writer


* We suggest to not execute this script since it takes too much time (2days in our case), we gonna offer our file results for this.  Go to the step of Analysis of data

In [3]:
# Twitter API keys (we let our keys if you wanna try the script without creting yours) 
consumer_key = "FZS588yAymChm15x5ZDrSP9me"
consumer_secret = "k9gWe709W25BlIoX0nuJzkiwDLpgJr0J6OhZToWnRPpsKRlJuN"
access_token = "764250739-fjrc9XrB5Wrd61Zk9axN7hHsgBwn0zzbLMJN3vmy"
access_token_secret = "YbWQQIn40lSQceuUbcUqVU0MxQL2CvBkx8ZqqPpRoFPpi"

In [ ]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# set access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)
# calling the api
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
#get list of writers
writers=dtweet["writer"].unique()

#script
NpFollowers={}
for wr in writers:
    try:
        user = api.get_user(wr)
        NpFollowers[wr]=user.followers_count
    except:
        NpFollowers[wr]=0

Some writers doesn't exist anymore, if its the case we put just 0 on their number of followers

In [26]:
#save into a file
csv_columns = ['writer','NbFowers']
try:
    with open('followers.csv', 'w') as csv_file:  
        writer = csv.writer(csv_file)
        writer.writerow(csv_columns)
        for key, value in NpFollowers.items():
            writer.writerow([key, value])
except IOError:
    print("I/O error")

## Anaysis of dataset

In [4]:
dcomany.head(10)

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc
2,GOOGL,Google Inc
3,AMZN,Amazon.com
4,TSLA,Tesla Inc
5,MSFT,Microsoft


In [5]:
dcomany.count()

ticker_symbol    6
company_name     6
dtype: int64

We can see the Google Inc has two symbols, se we gonna groupby the name of company and not the symbol

In [6]:
dcomany_tweets

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL
...,...,...
4336440,1212158772015034369,TSLA
4336441,1212159099632267268,TSLA
4336442,1212159184931717120,TSLA
4336443,1212159838882533376,TSLA


In [7]:
dcomany_tweets.count()

tweet_id         4336445
ticker_symbol    4336445
dtype: int64

In [8]:
dtweet

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
...,...,...,...,...,...,...,...
3717959,1212159765914079234,TEEELAZER,1577836383,That $SPY $SPX puuump in the last hour was the...,1,0,6
3717960,1212159838882533376,ShortingIsFun,1577836401,In 2020 I may start Tweeting out positive news...,0,0,1
3717961,1212160015332728833,Commuternyc,1577836443,Patiently Waiting for the no twitter sitter tw...,0,0,5
3717962,1212160410692046849,MoriaCrypto,1577836537,I don't discriminate. I own both $aapl and $ms...,1,0,1


## Merge data files

* First we read the followers file

In [9]:
followers = pd.read_csv("data/followers.csv")

In [10]:
followers

,writer,NbFowers
0,VisualStockRSRC,2005
1,KeralaGuy77,406
2,DozenStocks,835
3,ShowDreamCar,13109
4,i_Know_First,29803
...,...,...
140127,MacroTeller,10
140128,tradeoilstocks,576
140129,nuancerocket,92
140130,vistapglobal,9


In [11]:
followers.count()

writer      140131
NbFowers    140132
dtype: int64

* Merge files

In [12]:
#original dataset files
compa=pd.merge(dcomany_tweets, dcomany, on="ticker_symbol")
data=pd.merge(compa, dtweet, on="tweet_id")

In [13]:
#Merge our data with the followers generated data
data=pd.merge(data, followers, on="writer")

In [14]:
data

,tweet_id,ticker_symbol,company_name,writer,post_date,body,comment_num,retweet_num,like_num,NbFowers
0,550803612197457920,AAPL,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,0
1,550803612197457920,AMZN,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,0
2,550803610825928706,AAPL,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,0
3,550803610825928706,GOOGL,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,0
4,550803610825928706,AMZN,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4336440,1212075867385532417,TSLA,Tesla Inc,Keranrin,1577816380,My heart says ride $TSLA to 1000 but brain say...,0,0,1,17
4336441,1212091822106972160,TSLA,Tesla Inc,lauroalvarado,1577820184,$TSLA looks like the fuckin Niners at the las ...,0,0,0,496
4336442,1212111205558562817,TSLA,Tesla Inc,lorraine_matt,1577824805,$TSLA happened so fast. Another one I missed a...,0,0,0,64
4336443,1212120655434530816,TSLA,Tesla Inc,raycallaway1,1577827059,Finished the year with a =) trading with the b...,1,2,6,250


Now we can see the number of followers of each tweet writer

## Extruct the hashtags from the body tweet

In [15]:
data['hashtags'] = data["body"].str.findall(r"#(\w+)")
data['hashtags']=data['hashtags'].to_list()

In [16]:
data

,tweet_id,ticker_symbol,company_name,writer,post_date,body,comment_num,retweet_num,like_num,NbFowers,hashtags
0,550803612197457920,AAPL,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,0,"[TOPTICKERTWEETS, sentiquant]"
1,550803612197457920,AMZN,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,0,"[TOPTICKERTWEETS, sentiquant]"
2,550803610825928706,AAPL,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,0,"[SENTISHIFTUP, sentiquant]"
3,550803610825928706,GOOGL,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,0,"[SENTISHIFTUP, sentiquant]"
4,550803610825928706,AMZN,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,0,"[SENTISHIFTUP, sentiquant]"
...,...,...,...,...,...,...,...,...,...,...,...
4336440,1212075867385532417,TSLA,Tesla Inc,Keranrin,1577816380,My heart says ride $TSLA to 1000 but brain say...,0,0,1,17,[]
4336441,1212091822106972160,TSLA,Tesla Inc,lauroalvarado,1577820184,$TSLA looks like the fuckin Niners at the las ...,0,0,0,496,[]
4336442,1212111205558562817,TSLA,Tesla Inc,lorraine_matt,1577824805,$TSLA happened so fast. Another one I missed a...,0,0,0,64,[]
4336443,1212120655434530816,TSLA,Tesla Inc,raycallaway1,1577827059,Finished the year with a =) trading with the b...,1,2,6,250,[]


The list of hashtags is also added to the dataset

* remove the column that we will not use

In [17]:
column_final=['tweet_id','company_name','writer','NbFowers','post_date','body','hashtags']

In [18]:
final_data=data[column_final]

In [19]:
final_data

,tweet_id,company_name,writer,NbFowers,post_date,body,hashtags
0,550803612197457920,apple,SentiQuant,0,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,"[TOPTICKERTWEETS, sentiquant]"
1,550803612197457920,Amazon.com,SentiQuant,0,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,"[TOPTICKERTWEETS, sentiquant]"
2,550803610825928706,apple,SentiQuant,0,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,"[SENTISHIFTUP, sentiquant]"
3,550803610825928706,Google Inc,SentiQuant,0,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,"[SENTISHIFTUP, sentiquant]"
4,550803610825928706,Amazon.com,SentiQuant,0,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,"[SENTISHIFTUP, sentiquant]"
...,...,...,...,...,...,...,...
4336440,1212075867385532417,Tesla Inc,Keranrin,17,1577816380,My heart says ride $TSLA to 1000 but brain say...,[]
4336441,1212091822106972160,Tesla Inc,lauroalvarado,496,1577820184,$TSLA looks like the fuckin Niners at the las ...,[]
4336442,1212111205558562817,Tesla Inc,lorraine_matt,64,1577824805,$TSLA happened so fast. Another one I missed a...,[]
4336443,1212120655434530816,Tesla Inc,raycallaway1,250,1577827059,Finished the year with a =) trading with the b...,[]


In [20]:
final_data

,tweet_id,company_name,writer,NbFowers,post_date,body,hashtags
0,550803612197457920,apple,SentiQuant,0,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,"[TOPTICKERTWEETS, sentiquant]"
1,550803612197457920,Amazon.com,SentiQuant,0,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,"[TOPTICKERTWEETS, sentiquant]"
2,550803610825928706,apple,SentiQuant,0,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,"[SENTISHIFTUP, sentiquant]"
3,550803610825928706,Google Inc,SentiQuant,0,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,"[SENTISHIFTUP, sentiquant]"
4,550803610825928706,Amazon.com,SentiQuant,0,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,"[SENTISHIFTUP, sentiquant]"
...,...,...,...,...,...,...,...
4336440,1212075867385532417,Tesla Inc,Keranrin,17,1577816380,My heart says ride $TSLA to 1000 but brain say...,[]
4336441,1212091822106972160,Tesla Inc,lauroalvarado,496,1577820184,$TSLA looks like the fuckin Niners at the las ...,[]
4336442,1212111205558562817,Tesla Inc,lorraine_matt,64,1577824805,$TSLA happened so fast. Another one I missed a...,[]
4336443,1212120655434530816,Tesla Inc,raycallaway1,250,1577827059,Finished the year with a =) trading with the b...,[]


In [22]:
final_data.to_csv("tweets.csv", index=False)